In [1]:
%load_ext kedro.extras.extensions.ipython

                    WARNING  /home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/e ]8;id=577033;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=613981;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             xtras/extensions/ipython.py:18: DeprecationWarning:                                   
                             kedro.extras.extensions.ipython should be accessed only using the                     
                             alias kedro.ipython. The unaliased name will be removed in Kedro                      
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=851294;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=795413;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/06/24 13:25:30] INFO     Kedro project raftel                                                   ]8;id=932360;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=192243;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=738861;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=76418;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=507463;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=557563;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [2]:
%reload_kedro

                    INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=54030;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=395279;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/06/24 13:25:30] INFO     Kedro project raftel                                                   ]8;id=502258;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=861024;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=824594;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=792767;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=972747;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=78051;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [4]:
aras_a = catalog.load("aras_a@spark")

                    INFO     Loading data from 'aras_a@spark' (SparkDataSet)...                 ]8;id=511905;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=78618;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [6]:
window = Window.orderBy("SEC")

activities_duration = aras_a.withColumn(
    "SC", 
    F.when(
        (
            (F.lag("LR1", 1).over(window).isNull()) | 
            (F.lag("LR2", 1).over(window).isNull())
        ), 
        True
    ).when(
        (
            (F.col("LR1") != F.lag("LR1", 1).over(window)) | 
            (F.col("LR2") != F.lag("LR2", 1).over(window))
        ), 
        True
    ).otherwise(False)
).filter(
    F.col("SC") == True
).orderBy(
    "DAY", 
    "SEC"
).withColumn(
    "DURATION",
    (F.lag("SEC", -1).over(window) - F.col("SEC"))
).select(
    "LR1",
    "LR2",
    "LABEL",
    "DURATION"
)

In [7]:
activities_duration.groupby("LR1", "LR2").agg(F.max("DURATION").alias("AvgDuration") / 60).filter((F.col("LR1")==14) & (F.col("LR2")==25)).show()

24/02/06 13:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:27:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 1

+---+---+-----------------------------------+
|LR1|LR2|(max(DURATION) AS AvgDuration / 60)|
+---+---+-----------------------------------+
| 14| 25|                0.11666666666666667|
+---+---+-----------------------------------+



In [19]:
aras_a.filter((F.col("LR1")==14) & (F.col("LR2")==25)).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+------+
|Ph1|Ph2|Ir1|Fo1|Fo2|Di3|Di4|Ph3|Ph4|Ph5|Ph6|Co1|Co2|Co3|So1|So2|Di1|Di2|Te1|Fo3|LR1|LR2| DAY|   SEC|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+------+
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 14| 25|10.0|844108|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 14| 25|10.0|844109|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 14| 25|10.0|844110|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0| 14| 25|10.0|844111|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 14| 25|10.0|844112|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 14| 25|10.0|844113|
|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [8]:
catalog.save("activities_duration@spark", activities_duration)

[02/06/24 13:28:00] INFO     Saving data to 'activities_duration@spark' (SparkDataSet)...       ]8;id=27136;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=849443;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#382\382]8;;\

24/02/06 13:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:28:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 13:28:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/06 1

In [39]:
aras_a.orderBy("DAY", "SEC").show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|Ph1|Ph2|Ir1|Fo1|Fo2|Di3|Di4|Ph3|Ph4|Ph5|Ph6|Co1|Co2|Co3|So1|So2|Di1|Di2|Te1|Fo3|LR1|LR2|DAY|SEC|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  0|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  1|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  2|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  3|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  4|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  5|
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 12| 17|1.0|  6|
|  0|  0|  0|  0|  0